In [1]:
import pandas as pd
import os
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import recall_score, confusion_matrix
from itertools import combinations
from math import *


In [2]:
# cat = ['V_cat','A_cat']
classes = ['L', 'M', 'H']
devel_V_set = ['BoW','frozen-bert-gmax','frozen-bert-pos-fuse-rnnatt','frozen-bert-rnnatt','fused','NGD_TF_IDF_1500','PMI_without_log_method_1500','TF-IDF']
devel_A_set = ['BoW','frozen-bert-gmax','frozen-bert-pos-fuse-rnnatt','frozen-bert-rnnatt','fused','NGD_with_log_method_1200','PMI_without_log_method_1800','TF-IDF']
df = pd.read_csv('transcripted_text.st1_st2_st3.all.csv')
df_devel = df[df['partition'].isin(["devel"])]
V_list = df_devel['V_cat'].values.tolist()
A_list = df_devel['A_cat'].values.tolist()

In [3]:
def ArithmeticMean(data):
    len_d = float(len(data))
    result = sum(data) / len_d
    return result

def GeometricMean(data):
    len_d = len(data)
    product = 1.0
    for i in range(len_d):
        product = product * data[i]
    # the next line is equal to calculate the n-root
    result = product ** (1.0/len_d)
    return result

def HarmonicMean(data):
    len_d = len(data)
    x = [1.0/data[i] for i in range(len_d)]
    result = float(len_d) / sum(x)
    return result

def QuadraticMean(data):
    len_d = len(data)
    x = [data[i] * data[i] for i in range(len_d)]
    result = sqrt(sum(x) / float(len_d))
    return result

In [4]:
def readcsv(feature,cat):
    df = pd.read_csv(cat+'_mean/'+feature + '.csv')
    df_list = df.values.tolist()
    return df_list


prediction_list_L = []
prediction_list_M = []
prediction_list_H = []
def fusion(input_set):
    result_dict = dict()
#     result_dict_A = dict()
    if input_set == devel_V_set:
        cat = 'V_cat'
        test_list = V_list
    else:
        cat = 'A_cat'
        test_list = A_list
    print(cat)
    for num in range(1,len(input_set)):
        new_list = list(combinations(input_set,num))
        for sets in new_list:
            for prediction in sets:
                vote_list = readcsv(prediction,cat)
                pred_L = [i[1] for i in vote_list]
                pred_M = [i[2] for i in vote_list]
                pred_H = [i[3] for i in vote_list]
                prediction_list_L.append(pred_L)
                prediction_list_M.append(pred_M)
                prediction_list_H.append(pred_H)
            new_prediction = []
            for index in range(0,87):
                single_pred_list_L = []
                single_pred_list_M = []
                single_pred_list_H = []
                for prediction in prediction_list_L:
                    single_pred_list_L.append(prediction[index])
                for prediction in prediction_list_M:
                    single_pred_list_M.append(prediction[index])
                for prediction in prediction_list_H:
                    single_pred_list_H.append(prediction[index])
                mean_list =[ArithmeticMean(single_pred_list_L),ArithmeticMean(single_pred_list_M),ArithmeticMean(single_pred_list_H)]
                label = mean_list.index(max(mean_list))
                if label == 0:
                    new_prediction.append('L')
                if label == 1:
                    new_prediction.append('M')
                if label == 2:
                    new_prediction.append('H')
                    
#             print(new_prediction)
            r = recall_score(test_list, new_prediction,labels=classes, average='macro')*100
            result_dict[sets] = r
    max_list = []
    max_value = max(result_dict.values())  # 利用 max 函数先找到最大 value
    for m, n in result_dict.items():       # 遍历字典一遍找对应的 key 值
        if n == max_value:
            max_list.append(m)
#     print(max_list)
    return max_list, max_value


print(fusion(devel_V_set))
print(fusion(devel_A_set))

V_cat
([('TF-IDF',), ('BoW', 'PMI_without_log_method_1500'), ('BoW', 'TF-IDF'), ('PMI_without_log_method_1500', 'TF-IDF'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt'), ('BoW', 'NGD_TF_IDF_1500', 'PMI_without_log_method_1500'), ('BoW', 'NGD_TF_IDF_1500', 'TF-IDF'), ('BoW', 'PMI_without_log_method_1500', 'TF-IDF'), ('frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'frozen-bert-rnnatt'), ('frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'fused'), ('frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'PMI_without_log_method_1500'), ('fused', 'PMI_without_log_method_1500', 'TF-IDF'), ('NGD_TF_IDF_1500', 'PMI_without_log_method_1500', 'TF-IDF'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'frozen-bert-rnnatt'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'fused'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'PMI_without_log_method_1500'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'TF-IDF'), ('BoW', 'frozen-bert-po

([('frozen-bert-pos-fuse-rnnatt', 'NGD_with_log_method_1200', 'PMI_without_log_method_1800'), ('frozen-bert-pos-fuse-rnnatt', 'NGD_with_log_method_1200', 'TF-IDF'), ('frozen-bert-pos-fuse-rnnatt', 'PMI_without_log_method_1800', 'TF-IDF'), ('frozen-bert-rnnatt', 'fused', 'NGD_with_log_method_1200'), ('frozen-bert-rnnatt', 'fused', 'PMI_without_log_method_1800'), ('frozen-bert-rnnatt', 'fused', 'TF-IDF'), ('frozen-bert-rnnatt', 'NGD_with_log_method_1200', 'PMI_without_log_method_1800'), ('frozen-bert-rnnatt', 'NGD_with_log_method_1200', 'TF-IDF'), ('frozen-bert-rnnatt', 'PMI_without_log_method_1800', 'TF-IDF'), ('fused', 'NGD_with_log_method_1200', 'PMI_without_log_method_1800'), ('fused', 'NGD_with_log_method_1200', 'TF-IDF'), ('fused', 'PMI_without_log_method_1800', 'TF-IDF'), ('NGD_with_log_method_1200', 'PMI_without_log_method_1800', 'TF-IDF'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 'frozen-bert-rnnatt'), ('BoW', 'frozen-bert-gmax', 'frozen-bert-pos-fuse-rnnatt', 